Representación vectorial de texto TF-IDF

In [5]:
import re
import pandas as pd
import numpy as np

corpus = {'D1': 'in the new york times in',
          'D2': 'the new york post',
          'D3': 'the los angeles times'}

corpus = pd.DataFrame.from_dict(corpus, orient='index', columns=['texto'])

corpus

,texto
D1,in the new york times in
D2,the new york post
D3,the los angeles times


TF= Frecuencia de los términos, contar el número de veces que ocurre una palabra / divido por las palabras de ese documento.
Contar el número de ocurrencias de una palabra en un documento, dividio por el número de palabras en ese documento.

In [95]:
corpus.texto

D1    in the new york times in
D2           the new york post
D3       the los angeles times
Name: texto, dtype: object

In [96]:
corpus.texto.apply(lambda val:"*"+val)

D1    *in the new york times in
D2           *the new york post
D3       *the los angeles times
Name: texto, dtype: object

In [97]:
corpus.texto.apply(lambda val: val.split())

D1    [in, the, new, york, times, in]
D2             [the, new, york, post]
D3         [the, los, angeles, times]
Name: texto, dtype: object

In [98]:
corpus.texto.apply(lambda val: len(val.split())) #cuantas palabras tiene cada documento

D1    6
D2    4
D3    4
Name: texto, dtype: int64

In [99]:
corpus['d']=corpus.texto.apply(lambda val: len(val.split()))
corpus

,texto,d
D1,in the new york times in,6
D2,the new york post,4
D3,the los angeles times,4


In [100]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
bow_re=count_vect.fit_transform(corpus['texto'].values) #Cuantas veces ocurre una palabra dentro de un documento
bow_re.toarray() #bolsa de palabras

array([[0, 2, 0, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 0]], dtype=int64)

In [101]:
tf=pd.DataFrame(bow_re.toarray())
tf

,0,1,2,3,4,5,6,7
0,0,2,0,1,0,1,1,1
1,0,0,0,1,1,1,0,1
2,1,0,1,0,0,1,1,0


In [102]:
tf.columns=count_vect.get_feature_names() #Arroja el vocabulario con el orden de la matriz
tf.index=corpus.index
tf

,angeles,in,los,new,post,the,times,york
D1,0,2,0,1,0,1,1,1
D2,0,0,0,1,1,1,0,1
D3,1,0,1,0,0,1,1,0


In [103]:
tf=tf.T #transpuesta de la matriz, #in ocurrio dos veces, contar el número de cada palabra en el documento
tf

,D1,D2,D3
angeles,0,0,1
in,2,0,0
los,0,0,1
new,1,1,0
post,0,1,0
the,1,1,1
times,1,0,1
york,1,1,0


In [105]:
tf_=tf.div(corpus['d'], axis=1).round(3) #A través de las columnas 1 valor max que puede tener tf si Hola esta 3 y la cantidad de palabras es 3
#documento tamaño cero, el cod no funciona.
tf_

,D1,D2,D3
angeles,0.000,0.00,0.25
in,0.333,0.00,0.00
los,0.000,0.00,0.25
new,0.167,0.25,0.00
post,0.000,0.25,0.00
the,0.167,0.25,0.25
times,0.167,0.00,0.25
york,0.167,0.25,0.00


DF: Document Frequency 
La frencuencia de terminos es mas alta para palabras frecuentemente usadas en el documentos, es el numero de documentos que tiene esa palabra sobre el numero total de documentos

In [106]:
corpus['texto'].apply(lambda val: ":)"+val )

D1    :)in the new york times in
D2           :)the new york post
D3       :)the los angeles times
Name: texto, dtype: object

In [107]:
corpus['texto'].apply(lambda val: "york" in val )

D1     True
D2     True
D3    False
Name: texto, dtype: bool

In [108]:
corpus['texto'].apply(lambda val: "york" in val ).sum() #york esta en dos documentos, eso hace el bucle

2

In [109]:
corpus.shape [0] #Cuantos documentos tenenmos en el corpus, tupla fila, columna

3

In [110]:
df={}
for palabra in count_vect.get_feature_names(): #Esta la palabra york en el documento?
    suma=corpus['texto'].apply(lambda val: palabra in val ).sum()
    df[palabra] = suma
#recorro cada palabra, cuantas veces aparece en el documento

In [111]:
df = pd.DataFrame.from_dict(df, orient="index", columns=['doc_count'])

N = corpus.shape[0]

df['df'] = df['doc_count']/N #calculo de DF

df

,doc_count,df
angeles,1,0.333333
in,1,0.333333
los,1,0.333333
new,2,0.666667
post,1,0.333333
the,3,1.000000
times,2,0.666667
york,2,0.666667


La frecuencia en documentos es más alta para palabras usadas en muchos documentos
por otro lado, una palanra especifica a algun documento va a tener frecuencia de termino muy baja
Como el objetivo es distinguir un documento del otro, queremos 

TF-IDF

In [112]:
tf

,D1,D2,D3
angeles,0,0,1
in,2,0,0
los,0,0,1
new,1,1,0
post,0,1,0
the,1,1,1
times,1,0,1
york,1,1,0


In [113]:
df

,doc_count,df
angeles,1,0.333333
in,1,0.333333
los,1,0.333333
new,2,0.666667
post,1,0.333333
the,3,1.000000
times,2,0.666667
york,2,0.666667


In [114]:
df['idf']=1/df['df']
df['log_idf']=np.log10(df['idf'])
df

,doc_count,df,idf,log_idf
angeles,1,0.333333,3.0,0.477121
in,1,0.333333,3.0,0.477121
los,1,0.333333,3.0,0.477121
new,2,0.666667,1.5,0.176091
post,1,0.333333,3.0,0.477121
the,3,1.000000,1.0,0.000000
times,2,0.666667,1.5,0.176091
york,2,0.666667,1.5,0.176091


In [117]:
tf_

,D1,D2,D3
angeles,0.000,0.00,0.25
in,0.333,0.00,0.00
los,0.000,0.00,0.25
new,0.167,0.25,0.00
post,0.000,0.25,0.00
the,0.167,0.25,0.25
times,0.167,0.00,0.25
york,0.167,0.25,0.00


In [119]:
tdidf=df.join(tf_) #tf_ final
tdidf

,doc_count,df,idf,log_idf,D1,D2,D3
angeles,1,0.333333,3.0,0.477121,0.000,0.00,0.25
in,1,0.333333,3.0,0.477121,0.333,0.00,0.00
los,1,0.333333,3.0,0.477121,0.000,0.00,0.25
new,2,0.666667,1.5,0.176091,0.167,0.25,0.00
post,1,0.333333,3.0,0.477121,0.000,0.25,0.00
the,3,1.000000,1.0,0.000000,0.167,0.25,0.25
times,2,0.666667,1.5,0.176091,0.167,0.00,0.25
york,2,0.666667,1.5,0.176091,0.167,0.25,0.00


In [121]:
tdidf["tdidf_d1"]=tdidf['D1']*tdidf['log_idf']
tdidf["tdidf_d2"]=tdidf['D2']*tdidf['log_idf']
tdidf["tdidf_d3"]=tdidf['D3']*tdidf['log_idf']
tdidf #Por cada palabra y por cada documentos

,doc_count,df,idf,log_idf,D1,D2,D3,tdidf_d1,tdidf_d2,tdidf_d3
angeles,1,0.333333,3.0,0.477121,0.000,0.00,0.25,0.000000,0.000000,0.119280
in,1,0.333333,3.0,0.477121,0.333,0.00,0.00,0.158881,0.000000,0.000000
los,1,0.333333,3.0,0.477121,0.000,0.00,0.25,0.000000,0.000000,0.119280
new,2,0.666667,1.5,0.176091,0.167,0.25,0.00,0.029407,0.044023,0.000000
post,1,0.333333,3.0,0.477121,0.000,0.25,0.00,0.000000,0.119280,0.000000
the,3,1.000000,1.0,0.000000,0.167,0.25,0.25,0.000000,0.000000,0.000000
times,2,0.666667,1.5,0.176091,0.167,0.00,0.25,0.029407,0.000000,0.044023
york,2,0.666667,1.5,0.176091,0.167,0.25,0.00,0.029407,0.044023,0.000000


In [125]:
tdidf[["tdidf_d1","tdidf_d2","tdidf_d3"]]

,tdidf_d1,tdidf_d2,tdidf_d3
angeles,0.000000,0.000000,0.119280
in,0.158881,0.000000,0.000000
los,0.000000,0.000000,0.119280
new,0.029407,0.044023,0.000000
post,0.000000,0.119280,0.000000
the,0.000000,0.000000,0.000000
times,0.029407,0.000000,0.044023
york,0.029407,0.044023,0.000000


In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect=TfidfVectorizer()
corpus

,texto,d
D1,in the new york times in,6
D2,the new york post,4
D3,the los angeles times,4


In [130]:
tfidf = tfidf_vect.fit_transform(corpus['texto'].values)
tfidf

<3x8 sparse matrix of type '<class 'numpy.float64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [132]:
tfidf_matrix = pd.DataFrame(tfidf.toarray(), columns=tfidf_vect.get_feature_names())
tfidf_matrix.index = corpus.index

tfidf_matrix.T.round(3) #Los números son diferentes, ya que parte de log agregaron otras cosas para mejorar el algoritmo
#Consultar más información

,D1,D2,D3
angeles,0.000,0.000,0.584
in,0.811,0.000,0.000
los,0.000,0.000,0.584
new,0.308,0.480,0.000
post,0.000,0.632,0.000
the,0.239,0.373,0.345
times,0.308,0.000,0.445
york,0.308,0.480,0.000


Que tan importante es esa palabra para ese documentos, los y angeles tiene el mismo peso para el documento 3 los angeles tiene un 58% de importacia para el documentos.

Medidas de similitud, Qué tan parecidos son los documentos?

In [6]:
n1 = "La compañía Boring de Elon Musk construirá una conexión de alta velocidad en el aeropuerto de Chicago"
n2 = "La compañía Boring de Elon Musk construirá un enlace de alta velocidad al aeropuerto de Chicago"
n3 = "La empresa Boring de Elon Musk aprobó la construcción del tránsito de alta velocidad entre el centro de Chicago y el aeropuerto O'Hare."
n4 = "Tanto la manzana como la naranja son frutas"

corpus = {'n1': n1,
          'n2': n2,
          'n3': n3,
          'n4': n4}

corpus = pd.DataFrame.from_dict(corpus, orient='index', columns=['texto'])
corpus

,texto
n1,La compañía Boring de Elon Musk construirá una...
n2,La compañía Boring de Elon Musk construirá un ...
n3,La empresa Boring de Elon Musk aprobó la const...
n4,Tanto la manzana como la naranja son frutas


Preprocesado

In [7]:
import re
from nltk.corpus import stopwords
stopwords_sp=stopwords.words('spanish') #Mejor formato es una lista de documentos, lista de cadenas

def pre_procesado(texto): #Recibe una cadena de texto y devuelve una cadena de texto
    texto=texto.lower()
    texto=re.sub(r"[\W\d_]+",' ',texto)
    texto=texto.split() #tokenizacion
    texto=[palabra for palabra in texto if palabra not in stopwords_sp]
    texto=" ".join(texto)
    return texto

pre_procesado("Hola 1309487;;")

'hola'

In [8]:
corpus

,texto
n1,La compañía Boring de Elon Musk construirá una...
n2,La compañía Boring de Elon Musk construirá un ...
n3,La empresa Boring de Elon Musk aprobó la const...
n4,Tanto la manzana como la naranja son frutas


In [9]:
corpus['pp']=corpus['texto'].apply(lambda val: pre_procesado(val))
corpus

,texto,pp
n1,La compañía Boring de Elon Musk construirá una...,compañía boring elon musk construirá conexión ...
n2,La compañía Boring de Elon Musk construirá un ...,compañía boring elon musk construirá enlace al...
n3,La empresa Boring de Elon Musk aprobó la const...,empresa boring elon musk aprobó construcción t...
n4,Tanto la manzana como la naranja son frutas,manzana naranja frutas


In [10]:
corpus['pp'].values

array(['compañía boring elon musk construirá conexión alta velocidad aeropuerto chicago',
       'compañía boring elon musk construirá enlace alta velocidad aeropuerto chicago',
       'empresa boring elon musk aprobó construcción tránsito alta velocidad centro chicago aeropuerto hare',
       'manzana naranja frutas'], dtype=object)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()
tfidf = tfidf_vec.fit_transform(corpus['pp'].values)  #Aprende y luego transforma de los aprendiste

tfidf_matrix = pd.DataFrame(tfidf.toarray())
tfidf_matrix.columns = tfidf_vec.get_feature_names()
tfidf_matrix.index = corpus.index

tfidf_matrix = tfidf_matrix.T

tfidf_matrix


,n1,n2,n3,n4
aeropuerto,0.282775,0.282775,0.214535,0.00000
alta,0.282775,0.282775,0.214535,0.00000
aprobó,0.000000,0.000000,0.336111,0.00000
boring,0.282775,0.282775,0.214535,0.00000
centro,0.000000,0.000000,0.336111,0.00000
chicago,0.282775,0.282775,0.214535,0.00000
compañía,0.349284,0.349284,0.000000,0.00000
conexión,0.443022,0.000000,0.000000,0.00000
construcción,0.000000,0.000000,0.336111,0.00000
construirá,0.349284,0.349284,0.000000,0.00000


In [12]:
tfidf_matrix.T #Las filas son los documentos

,aeropuerto,alta,aprobó,boring,centro,chicago,compañía,conexión,construcción,construirá,elon,empresa,enlace,frutas,hare,manzana,musk,naranja,tránsito,velocidad
n1,0.282775,0.282775,0.000000,0.282775,0.000000,0.282775,0.349284,0.443022,0.000000,0.349284,0.282775,0.000000,0.000000,0.00000,0.000000,0.00000,0.282775,0.00000,0.000000,0.282775
n2,0.282775,0.282775,0.000000,0.282775,0.000000,0.282775,0.349284,0.000000,0.000000,0.349284,0.282775,0.000000,0.443022,0.00000,0.000000,0.00000,0.282775,0.00000,0.000000,0.282775
n3,0.214535,0.214535,0.336111,0.214535,0.336111,0.214535,0.000000,0.000000,0.336111,0.000000,0.214535,0.336111,0.000000,0.00000,0.336111,0.00000,0.214535,0.00000,0.336111,0.214535
n4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000,0.57735,0.000000,0.57735,0.000000,0.000000


Distancia euclidiana

In [159]:
from sklearn.metrics.pairwise import euclidean_distances

dist_euc=euclidean_distances(tfidf_matrix.T.values)
dist_euc=pd.DataFrame(dist_euc, columns=tfidf_matrix.columns, index=tfidf_matrix.columns)

print(dist_euc)
corpus

          n1        n2        n3        n4
n1  0.000000  0.626528  1.072701  1.414214
n2  0.626528  0.000000  1.072701  1.414214
n3  1.072701  1.072701  0.000000  1.414214
n4  1.414214  1.414214  1.414214  0.000000


,texto,pp
n1,La compañía Boring de Elon Musk construirá una...,compañía boring elon musk construirá conexión ...
n2,La compañía Boring de Elon Musk construirá un ...,compañía boring elon musk construirá enlace al...
n3,La empresa Boring de Elon Musk aprobó la const...,empresa boring elon musk aprobó construcción t...
n4,Tanto la manzana como la naranja son frutas,manzana naranja frutas


Distancia del conseno: Mide el ángulo que exite entre dos vectores

In [163]:
from sklearn.metrics.pairwise import cosine_distances #cosine_distances es cero si son más cercanos y el otro si es uno es más cercano

dist_cos=cosine_distances(tfidf_matrix.T.values)
dist_cos=pd.DataFrame(dist_cos, columns=tfidf_matrix.columns, index=tfidf_matrix.columns)
dist_cos #El angulo del documento 1 con el 4 es igual 1, por tanto no son parecidos.

,n1,n2,n3,n4
n1,0.000000,0.196269,0.575343,1.0
n2,0.196269,0.000000,0.575343,1.0
n3,0.575343,0.575343,0.000000,1.0
n4,1.000000,1.000000,1.000000,0.0


Distancia de Jaccard: Intersección de lo que tiene en común y su unión

In [168]:
def jaccard_distance(list1,list2):
    s1=set(list1)
    s2=set(list2)
    resultado=1-len(s1.intersection(s2))/len(s1.union(s2))
    return resultado

jaccard_distance(corpus.iloc[0]['pp'].split(),corpus.iloc[0]['pp'].split()) #Documentos 1 contra 1
jaccard_distance(corpus.iloc[0]['pp'].split(),corpus.iloc[1]['pp'].split())
jaccard_distance(corpus.iloc[0]['pp'].split(),corpus.iloc[2]['pp'].split())
jaccard_distance(corpus.iloc[0]['pp'].split(),corpus.iloc[3]['pp'].split())

1.0

Distancia de Levenshtein o distancia de edicion: Minimo de operaciones requeridas para transformar una cadena de caracteres en otra Se usa en los correctores de ortografia 

disntacia entre casa y calle es de tres porque se necesitan tres ediciones elementales para cambiar uno en el otro
casa cala sustitucion s por l
casa ca

In [169]:
import nltk

nltk.edit_distance(corpus.iloc[0]['pp'].split(), corpus.iloc[3]['pp'].split())

10